# Lab Audio/ Music Auto-Tagging multi-label

**WARNING: This jupiter notebook was created as part of Télécom-Paris teaching programme and is distributed only to its students. 
Any re-use, modification, distribution outside this framework or making it available (through github, colab or others) is forbidden.**

- Date: 2022/02/07
- Version: v0.1
- Author: geoffroy.peeters@telecom-paris.fr

## Objectives of this Lab:

[Magna-Tag-A-Tune](https://musicmachinery.com/2009/04/01/magnatagatune-a-new-research-data-set-for-mir/) is a dataset containing 25,863 music clips belonging to one of the 5223 songs, 445 albums and 230 artists. 
The clips span a broad range of genres like Classical, New Age, Electronica, Rock, Pop, World, Jazz, Blues, Metal, Punk, and more. 
Each audio clip has been annotated by humans playing the two-player online Tag-A-Tune game. 
The annotations include tags like ’singer’, ’no singer’, ’violin’, ’drums’, ’classical’, ’jazz’. 

From this large dataset, we extracted a smaller dataset (1000 music clips) corresponding to the annotation into **instrumentation**.
The labels of this dataset are highly unbalanced (some tags are much frequent than others).
In this lab, you will develop a system to perform the automatic estimation of these tags. Since tags can co-occur (they are not mutually exclusive), the problem is a **multi-label classification**.

## Dataset

This dataset is available as a .json file and a .zip file.
You will first upload both to your Google Drive in a folder named 'My Drive/_sound/_magnatagatune'.
You will first **mount your Google drive** in this notebook to allow accessing its storage.
For this lab, we will also run the code on the GPU of Google colab. Do not forget to set **execution type** to GPU.

## Feature extraction.
For each file, you will extract its audio representations/features using ```librosa```.
The representation we will use is the Log-Mel-Spectrogram (with 128 Mel bands), i.e. the Mel-Spectrogram in Log-scale. 
This representation is the standard inut representation for Deep Neural Network; it is close to the MFCC one but skipping the last DCT.
Note that we will use a $\log(1 + C x)$ with $C=10000$ instead of the standard $\log(x)$ function. This is to avoid over-representing very small values.

In order to avoid having to compute the audio representations/features each time we need them, we will store them as files in the Drive.

## Audio patches
We will then use a ConvNet to perform the classification of the file over time. 
For this the big matrix containing the audio representations./features is sliced over time as a succession of 2D patches.
These 2D patches are the inputs of our ```ConvNet```. 
A single file is therefore represented by a succession of 2D patches.

## Dataset and Dataloader 

Unlike scholar examples (using make_circle or MNIST) it is not possible to get all the data simultaneously in memory and then select the one needed for each mini-batch. Instead, we need to develop a ```generator```which will provide the data on-demand (get in memory only the data needed to process a mini-batch). We will do this using the very very convenient ```DataSet``` and ```DataLoader``` from pytorch.

## Deep Neural Network

We then train a ConvNet which takes as input the patches (by using the DataLoader) and return the probability of each tag. 

The network has as many outputs neurons as the number of tags.
Since we deal multi-label, neurons are independent of each others. 
We then use a sigmoid for each.
We minimize the sum of the Binary Cross Entropy (BCE) over neurons.
Note that in pytorch it is common to combine the sigmoid and the BCE into a new loss; here denoted ``BCEWithLogitsLoss``. 
In this case, the output of the network does not have any non-linear functions. 
In multi-label classification, for each class $c$, the number of negative examples $|y^{(i)}_c=0|$ is usually much larger than of positive examples $|y^{(i)}_c=1|$. It is therefore benefitial to give a higher weights to positive examples in ``BCEWithLogitsLoss``.

## Testing: tag-o-gram

Once train, we will apply our model to compute the tag-o-gram (likelihood of each tags over time) of an unknown file.



## At the end of this Lab, the student will be able to
- Use colab and connect it to Google Drive to work on/store remote data
- Process datasets directly from Colab
- Use librosa https://librosa.github.io/librosa/ to compute audio representations 
- Use DataSet and DataLoaders to get on the fly the necessary data for a batch
- Perform a multi-label task (tag an audio file into multi-labels) using a Deep Convolutional Neural Network in pytorch


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import glob
import os
import shutil
import json

import IPython
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')
import pprint as pp

import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

In [ ]:
# If necessary, install ffmpeg to allows mp3 decoding
#! pip install ffmpeg

# Parameters

In [ ]:
use_colab = True
use_gpu = True

do_magnatagatune1_gtzan2  = 1

patch_hop_frame = 17
patch_halfduration_frame = 34
data_ext = '.lms'
do_norm = True

dropout_prob = 0.2
nb_epoch = 50
do_model_train = True

do_student = True

do_class_method = 'multilabel'
subDIR = '/_magnatagatune/'
dataset_json = 'dataset_magna-tag-a-tune-sub-instr-100.json'
dataset_zip = 'dataset_magna-tag-a-tune.zip'
dataset_subDIR =  '/dataset_magna-tag-a-tune/'
labelname_dict_l = ['guitar', 'string', 'synth', 'drum', 'violin', 'piano', 'female', 'male', 'sitar', 'flute', 'harpsichord', 'harp']

# Set Google Drive

When you store locally data in Colab; these data will be removed at the end of your session.
In order to able to store definitely your data, you can connect Colab to your Google Drive and then store the data on it. It is done in the following way.

In [ ]:
from google.colab import drive
mnt_point = '/content/drive/'
drive.mount(mnt_point)
DIR = mnt_point + '/My Drive/_sound/' + subDIR

We now unzip the file containing the audio of the dataset.

In [ ]:
shutil.unpack_archive(DIR + dataset_zip, DIR)
len(glob.glob(DIR + dataset_subDIR + '*/*'))

# Load and prepare dataset

The dataset (relationships between audio files and tags) is described in a json file.
We load this file and convert the tags to a multi-hot-encoding.

In [ ]:
with open(DIR + dataset_json, 'r') as f:
    dataset_l = json.load(f)

if do_class_method=='multilabel':
  # ---- Convert tag to index of class
  for data in dataset_l:
      data['target'] = np.zeros(len(labelname_dict_l))
      for tag in data['tag']:
          if tag in labelname_dict_l:
              data['target'][ labelname_dict_l.index(tag) ] = 1

### Test

You should get the following output.
```
len(dataset_l):  1000
labelname_dict_l:  ['guitar', 'string', 'synth', 'drum', 'violin', 'piano', 'female', 'male', 'sitar', 'flute', 'harpsichord', 'harp']
{'album': 'J.S. Bach Solo Cantatas',
 'artist': 'American Bach Soloists',
 'clip_id': 6,
 'mp3_path': 'f/american_bach_soloists-j_s__bach_solo_cantatas-01-bwv54__i_aria-146-175.mp3',
 'original_url': 'http://he3.magnatune.com/all/01--BWV54%20-%20I%20Aria--ABS.mp3',
 'segmentEnd': 175,
 'segmentStart': 146,
 'tag': ['classical',
         'violin',
         'string',
         'classical',
         'opera',
         'violin',
         'baroque'],
 'target': array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 'title': 'BWV54 - I Aria',
 'track_number': 1,
 'url': 'http://www.magnatune.com/artists/albums/abs-solocantatas/'}
````


In [ ]:
print('len(dataset_l): ', len(dataset_l))
print('labelname_dict_l: ',  labelname_dict_l )
pp.pprint(dataset_l[1])  

# Audio features

We create here a function that, given an audio file as input, 
- load the audio file
- extract a given audio representation/ audio features
- store the results in the drive; we do so since if this only needs to be computed once. The second time we call this function on the same file, it will load the results from the drive instead of recomputing it.

We will use the ```librosa``` package for that.

In [ ]:
def F_do_standardization(data_m):
    """
    description:
      perform feature standardization (mean, std) over frames
    inputs:
      - data_m (nb_dim, nb_frame)
    outputs:
      - data_m (nb_dim, nb_frame)
    """
    if do_student:
        # --- START CODE HERE
        ...
        # --- STOP CODE HERE
          
    return data_m

In [ ]:
def F_log(data_m, C = 10000):
  """
  description:
    replacement to log function (to avoid emphasazing very small values)
  """
  return np.log(1 + C*data_m)

In [ ]:
def F_get_audio_features(audio_filename, data_ext=''):
    """
    description:
      compute Log-Mel-Sepctrogram audio features with n_mels=128, window length L_n = 2048, and hop-size STEP_n = 1024
    inputs:
      - audio_filename
      - data_ext
    outputs:
      - data_m (nb_dim, nb_frame): Log-Mel-Sepctrogram matrix
      - time_sec_v (nb_frame): corresponding time [in sec] of analysis windows
    """
    L_n = 2048
    STEP_n = 1024
    feature_filename = audio_filename + data_ext + '.npz'

    if not os.path.isfile(feature_filename):
        
        if do_student:
            # --- START CODE HERE
            ...
            data_m = ...
            time_sec_v = ...
            # --- STOP CODE HERE

        np.savez(feature_filename, data_m=data_m, time_sec_v=time_sec_v)
    else:
        all = np.load(feature_filename)
        data_m = all['data_m']
        time_sec_v = all['time_sec_v']

    return data_m, time_sec_v

### Test

We test the extraction of audio features on a single audio file. You should get the following output.

```
shape: (128, 628), min: 0.0, max: 9.236710548400879
```

<img src="https://perso.telecom-paristech.fr/gpeeters/doc/Lab_DL_MagnaTagATune_01.png">


In [ ]:
data_m, time_sec_v = F_get_audio_features(DIR + dataset_subDIR + dataset_l[900]['mp3_path'], data_ext);
plt.imshow(data_m, aspect='auto'); plt.colorbar()
print('shape: {}, min: {}, max: {}'.format(data_m.shape, np.min(data_m), np.max(data_m)))

# Patches

When we deal with audio files, the audio representations/features is a matrix $W$ which represents the values of the representation/feature (such as spectrum, CQT, MFCC, Chroma, Log-Mel-gram) for each time frame of the audio file.

If the analysis is performed with a 20ms hop size, and the total duration of the file is 30s; we therefore have a 1500 vectors; each vector represent the audio representation/feature at a specific time.

The matrix $W$ has the size ```(nb_feature, nb_time_frame)```.

To process it with a Convolutional Neural Network, we will split this big matrix into a set of small matrices $A_m$ (we name those ```patches```) where each $A_m$ represent a temporal slice of the big-matrix. 
The size of $A_m$ will be ```(nb_feature, size_of_patch)```.
$W$ will then be represented by a set of $A_m$. 
Each $A_m$ has the same size but represent a slice of the big-matrix starting at a different time $t$.

The two parameters we need to define are:
- the distance in frames between two successive patch ```patch_hop_frame```
- the width of the patch ```patch_halfduration_frame```

Given an audio file, the following function compute the start and end position of each patch $A_m$ within $W$.

In [ ]:
def F_slice_into_patches(patch_hop_frame, patch_halfduration_frame, patch_info_l, nb_frame, idx_file, label):
    """
    create structure for storing patch-based slides of the spectrogram
    """
    middle_frame = patch_halfduration_frame
    while middle_frame + patch_halfduration_frame < nb_frame:
        if do_student:
            # --- START CODE HERE
            ...
            # --- STOP CODE HERE

        patch_info_d = {'idx_file': idx_file, 
                        'start_frame': start_frame, 
                        'middle_frame': middle_frame, 
                        'end_frame': stop_frame, 
                        'target': label}
        patch_info_l.append(patch_info_d)

        middle_frame += patch_hop_frame
        return patch_info_l

### Test

You should get the following output.

```
[{'end_frame': 64,
  'idx_file': 333,
  'middle_frame': 32,
  'start_frame': 0,
  'target': [1, 0, 0, 1, 0]},
 {'end_frame': 80,
  'idx_file': 333,
  'middle_frame': 48,
  'start_frame': 16,
  'target': [1, 0, 0, 1, 0]},
 {'end_frame': 96,
  'idx_file': 333,
  'middle_frame': 64,
  'start_frame': 32,
  'target': [1, 0, 0, 1, 0]}]
  ```

In [ ]:
patch_info_l = F_slice_into_patches(patch_hop_frame=16, patch_halfduration_frame=32, patch_info_l=[], nb_frame=100, idx_file=333, label=[1,0,0, 1, 0])
pp.pprint(patch_info_l)

# Torch dataset
During the training of our network, we would like to have batches of data $x^{(i)}, y^{(i)}$ on the fly, without having to take care about where to read it from, how to format it and to check that all data have been processed before moving to the next epoch.

Pytorch provides a nice class for this: ``Dataset``.
This class has three mandatory methods:
- ``__init__``
- ``__len__``
- ``__getitem__``

Let's explain this from the end.
We instantiate the class using ``train_set = AudioDataSet(...)``.
We define one item of our dataset as one patch.
- Therefore ``len(train_set)`` should gives the number of patches of our dataset (which is the number of music tracks multiplied by the number of patches within a track). This is the goal of the ``__len__`` method.
- ``train_set[idx_patch]`` should give all the information for a given patch, i.e. $(x^{(i)}, y^{(i)}$ for this patch as well as the audio file and time position it is coming from. This is the goal of the ``__getitem__`` method.
- This means that ``__init__`` (the creator of the class) should store all the necessary information to allow ``__len__`` and ``__getitem__``.

Note that, in order to allow runing the model faster, we will store a part of the information directly on the GPU. We do so using ``.cuda()``.

We also add an option to the class to allow defining a dataset for the training and a dataset for the testing (here we do not split validation and testing).


In [ ]:
class AudioDataSet(Dataset):
    """AudioDataSet Dataset
    Args:
        train (bool, optional)
        audio_filename_l: list of audio files
        labelname_l: list of label names
    Attributes:
        - audio_filename_l: list of audio files
        - labelname_dict_l: list of the terms used for the labels
        - patch_info_l: list of dictionaries containing patches information
            'idx_file': identifier of the file used for this patch (reference to audio_filename_l)
            'start_frame': starting frame of the patch in the audio file
            'middle_frame': middle frame of the patch in the audio file
            'end_frame': end frame of the patch in the audio file
            'target': identifier of the target used for this patch (reference to labelname_dict_l)
    """

    audio_filename_l = []
    labelname_dict_l = []
    patch_info_l = []
    data_d = {}

    def __init__(self, train, dataset_l, labelname_dict_l):
        """
        """
        
        self.labelname_dict_l = labelname_dict_l
        
        # --- split into train and test
        if train: dataset_l = [dataset_l[idx] for idx in range(len(dataset_l)) if idx % 10 != 0]
        else:     dataset_l = [dataset_l[idx] for idx in range(len(dataset_l)) if idx % 10 == 0]
        
        self.audio_filename_l = [DIR + dataset_subDIR + data['mp3_path'] for data in dataset_l]
        
        self.patch_info_l = []
        self.data_d = {}
        
        if do_student:
            # --- START CODE HERE
            for idx_file in tqdm(range(len(self.audio_filename_l))):
                ...
                self.patch_info_l = F_slice_into_patches(patch_hop_frame, patch_halfduration_frame, self.patch_info_l, nb_frame, idx_file, label)
                self.data_d[self.audio_filename_l[idx_file]] = {'data_m': torch.from_numpy(data_m).float().cuda(), 'time_sec_v': time_sec_v}
            # --- STOP CODE HERE


    def __len__(self):
        """
        """
        return len(self.patch_info_l)



    def __getitem__(self, idx_patch):
        """
        return sample['idx_file','middle_frame','target','data']
        """
        audio_filename = self.audio_filename_l[ self.patch_info_l[idx_patch]['idx_file'] ]
        # --- Optimize: get directly from GPU
        data_m, time_sec_v = self.data_d[audio_filename]['data_m'], self.data_d[audio_filename]['time_sec_v']
        STEP_sec = np.mean(np.diff(time_sec_v))
        # --- get the patch
        data_m = data_m[:, self.patch_info_l[idx_patch]['start_frame']:self.patch_info_l[idx_patch]['end_frame']]
        
        # --- create output structure
        sample = {'idx_file': self.patch_info_l[idx_patch]['idx_file'],
                  'middle_frame': self.patch_info_l[idx_patch]['middle_frame'],
                  'target': self.patch_info_l[idx_patch]['target'],
                  'data': data_m,
                  'STEP_sec': STEP_sec
                  }
        return sample

Using this class, we instantiate a ``train_set`` and a ``test_set``.

In [ ]:
train_set = AudioDataSet(train=True, dataset_l=dataset_l, labelname_dict_l=labelname_dict_l)
test_set = AudioDataSet(train=False, dataset_l=dataset_l, labelname_dict_l=labelname_dict_l)

### Test

You should get the following output.

```
len(train_set):  31490
len(test_set):  3500
/content/drive//My Drive/_sound//_gtzan//dataset_gtzan//pop/pop.00065.wav
torch.Size([128, 68])
0.023219954648526078
duration of a patch in second: 1.5789569160997732
```

<img src="https://perso.telecom-paristech.fr/gpeeters/doc/Lab_DL_MagnaTagATune_02.png">

In [ ]:
print('len(train_set): ', len(train_set))
print('len(test_set): ', len(test_set))

idx_file = 50
print(train_set.audio_filename_l[idx_file])
print(train_set[idx_file]['data'].size())
print(train_set[idx_file]['STEP_sec'])
plt.imshow(train_set[idx_file]['data'].cpu().numpy()); plt.colorbar();

print('duration of a patch in second:', train_set[idx_file]['STEP_sec']*train_set[idx_file]['data'].size(1))

# Torch dataloader

```DataLoader```is a **very very convenient** way pytorch to provide to load on the fly the data necessary for a given mini-batch. It will simply pick-up from our DataSets the data necessary for the given mini-batch. We do not need to care about counting them or shuffling them. It does everything for us.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=False, num_workers=0)

### Test

You should get the following output.

```
torch.Size([32, 128, 68])
````


In [ ]:
one_mini_batch = next(iter(train_loader))
one_mini_batch['data'].size()

# DNN model

We now define the architecture of our Deep ConvNet to perform multi-label classification.
The network has the following architecture.

<img src="https://perso.telecom-paristech.fr/gpeeters/doc/Lab_DL_MagnaTagATune_03.png">

In [ ]:
class DeepConvNet(torch.nn.Module):
    
    def __init__(self, n_in_H, nb_class):
        """
        inputs:
        - n_in_H: input dimension (128 in the case of Log-Mel-Gram)
        - nb_class: output dimension (number of tags to be estimated)
        """

        super(DeepConvNet, self).__init__()
        
        if do_student:
            # --- START CODE HERE
            ...
            # --- STOP CODE HERE



    def forward(self, x):
        if do_student:
            # --- START CODE HERE
            ...
            # --- STOP CODE HERE

  
        return x

### Test

We instantiate the model, send it to the GPU and display its summary.
You should get the following output.

```
n_in_H:  128
cuda:0
torch.Size([2, 1, 128, 68])
l1_conv:  torch.Size([2, 256, 1, 64])
l1_pool:  torch.Size([2, 256, 1, 16])
l2_conv:  torch.Size([2, 256, 1, 12])
l2_pool: torch.Size([2, 256, 1, 6])
l3_conv:  torch.Size([2, 256, 1, 2])
l3_conv:  torch.Size([2, 256, 1, 1])
l4_fc:  torch.Size([2, 256])
l5_fc:  torch.Size([2, 256])
l6_fc:  torch.Size([2, 12])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
================================================================
            Conv2d-1           [-1, 256, 1, 64]         164,096
             PReLU-2           [-1, 256, 1, 64]               1
         MaxPool2d-3           [-1, 256, 1, 16]               0
           Dropout-4           [-1, 256, 1, 16]               0
            Conv2d-5           [-1, 256, 1, 12]         327,936
             PReLU-6           [-1, 256, 1, 12]               1
         MaxPool2d-7            [-1, 256, 1, 6]               0
           Dropout-8            [-1, 256, 1, 6]               0
            Conv2d-9            [-1, 256, 1, 2]         327,936
            PReLU-10            [-1, 256, 1, 2]               1
        MaxPool2d-11            [-1, 256, 1, 1]               0
          Dropout-12                  [-1, 256]               0
           Linear-13                  [-1, 256]          65,792
            PReLU-14                  [-1, 256]               1
          Dropout-15                  [-1, 256]               0
           Linear-16                  [-1, 256]          65,792
            PReLU-17                  [-1, 256]               1
          Dropout-18                  [-1, 256]               0
           Linear-19                   [-1, 12]           3,084
================================================================
Total params: 954,641
Trainable params: 954,641
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.03
Forward/backward pass size (MB): 0.41
Params size (MB): 3.64
Estimated Total Size (MB): 4.08
----------------------------------------------------------------
```

In [ ]:
# --- Instantiate the model
one_mini_batch = next(iter(train_loader))
n_in_H = one_mini_batch['data'].size(1)
print('n_in_H: ', n_in_H)
model = DeepConvNet(n_in_H=n_in_H, nb_class=len(labelname_dict_l))

# --- Send the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.cuda()

# --- Display the structure of the model
from torchsummary import summary
summary(model, input_size=(1, n_in_H, 68))

# Train the model

To train the model, we still need to define a loss to be minimized
- what is the loss for a multi-label problem ? 
- how to gives more weight to the positive classes ?

We also need to define an optimizer. We will use Adam with a learning rate of 1e-3.

In [ ]:
if do_student:
    # --- START CODE HERE
    criterion = ...
    optimizer = ...
    # --- STOP CODE HERE

### Test

Before launching the training, we test the whole pipeline on one batch.

In [ ]:
one_mini_batch = next(iter(train_loader))
x = one_mini_batch['data'][:,None,:,:]
y = one_mini_batch['target'].cuda()
y_hat = model(x)
loss = criterion(y_hat, y)

In [ ]:
import sys
sys.path.append(DIR)
import tools_score

In [ ]:
def train(train_loader, num_epoch):
    """
    train the model
    """
    # --- set model to train mode
    model.train()
    
    my_score = tools_score.Score(do_class_method)

    # --- loop over mini-batches
    for batch_idx, batch in enumerate(tqdm(train_loader)):
        # --- set gradient to zero (in order to avoid cumulating them over batch)
        model.zero_grad()        
        # --- we need to add the channel dimension to the input
        x = batch['data'][:,None,:,:]        
        y = batch['target'].cuda()
        y_hat = model(x)        
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()                
        
        my_score.add(loss, y_hat, y)
           
    my_score.print('Train', num_epoch)
    return my_score.get_summary()

In [ ]:
def test(test_loader, num_epoch):
    """
    test the model
    """
    # --- set model to eval mode
    model.eval()
    
    my_score = tools_score.Score(do_class_method)

    # --- loop over mini-batches
    for batch_idx, batch in enumerate(test_loader):
        # --- We need to add the channel dimension to the input
        x = batch['data'][:,None,:,:]
        y = batch['target'].cuda()
        y_hat = model(x)
        loss = criterion(y_hat, y)

        my_score.add(loss, y_hat, y)
    
    my_score.print('Test', num_epoch) 
    return my_score.get_summary()

#  Do training

In [ ]:
if do_model_train==True:
    score_train_l, score_test_l = [], []
    # --- loop over epochs
    for num_epoch in range(nb_epoch):    
        score_train = train(train_loader, num_epoch)
        score_test = test(test_loader, num_epoch)
        score_train_l.append(score_train)
        score_test_l.append(score_test)
    torch.save(model.state_dict(), DIR + 'model_torch' + data_ext)
else:
    model.load_state_dict(torch.load(DIR + 'model_torch' + data_ext))

## Display training/test curves

We now display the loss, accuracy, TPrate, TNrate for the training and test set.
You should get the following output.

<img src="https://perso.telecom-paristech.fr/gpeeters/doc/Lab_DL_MagnaTagATune_04.png">

In [ ]:
# --- Display loss and accuracy for train and test set
my_score = tools_score.Score(do_class_method)
my_score.plot_curve(score_train_l, score_test_l)

# Display results as tag-o-gram

Now that our model has been trained we will use it to create a tag-o-gram (tag likelihood over the patches of an audio file).

In [ ]:
def F_tag_o_gram(audio_file):
    """
    """
    if do_student:
        # --- START CODE HERE
        ...
        # --- STOP CODE HERE

    fig, axes = plt.subplots(1, 1, figsize=(16, 6))
    im = axes.imshow(y_hat_m.T, aspect='auto', cmap=plt.get_cmap('inferno'))
    axes.set_yticks(np.arange(0,len(test_set.labelname_dict_l)))
    axes.set_yticklabels(test_set.labelname_dict_l)
    axes.grid(True)
    fig.colorbar(im, orientation='vertical')

    return

### Test

We now test our tag-o-gram on an audio file.
You should get the following output.

```
{'album': 'CD2 The Sowebo Concert',
 'artist': 'Jay Kishor',
 'clip_id': 3649,
 'mp3_path': 'c/jay_kishor-cd2_the_sowebo_concert-01-raga_malkauns-146-175.mp3',
 'original_url': 'http://he3.magnatune.com/all/01-Raga%20Malkauns-Jay%20Kishor.mp3',
 'segmentEnd': 175,
 'segmentStart': 146,
 'tag': ['guitar', 'quiet', 'india'],
 'target': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'title': 'Raga Malkauns',
 'track_number': 1,
 'url': 'http://www.magnatune.com/artists/albums/kishor-sowebo2/'}
 ````

<img src="https://perso.telecom-paristech.fr/gpeeters/doc/Lab_DL_MagnaTagATune_05.png">

As you see, while the track has been tagged 'guitar', it actually contains a [sitar](https://en.wikipedia.org/wiki/Sitar); which has been correctly detected by our trained model.

In [ ]:
idx_file = 902
test_file = DIR + dataset_subDIR + dataset_l[idx_file]['mp3_path']
F_tag_o_gram(audio_file = test_file)
pp.pprint(dataset_l[idx_file])
IPython.display.Audio(test_file)